<a href="https://www.kaggle.com/code/yunasheng/eda-perth-housing-price-prediction?scriptVersionId=160476273" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd;import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly
from plotly.subplots import make_subplots
import os
from catboost import CatBoostClassifier,CatBoostRegressor
from sklearn.feature_selection import SelectKBest,f_regression
from xgboost import plot_importance,XGBClassifier,XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
import seaborn as sns
import shap
from scipy import stats

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff
from sklearn.base import BaseEstimator,TransformerMixin

%matplotlib inline
cmap = sns.diverging_palette(220,10,as_cmap=True)

import warnings
warnings.filterwarnings('ignore')

# Helper Functions located below

In [2]:
""" Various Helper Functions """

""" Plot Correlation Matrix """
def corrMat(df,id=False,figsize=(10,10)):
    
    corr_mat = df.corr().round(2)
    f,ax = plt.subplots(figsize=figsize)
    mask = np.triu(np.ones_like(corr_mat, dtype=np.bool))
    mask = mask[1:,:-1]
    corr = corr_mat.iloc[1:,:-1].copy()
    sns.heatmap(corr,mask=mask,vmin=-0.3,vmax=0.3,center=0,
               cmap='Blue',square=False,lw=2,annot=True,cbar=False)
    
"""Plot Correlation to Target Variable only"""
def corrMat2(df,target='demand',figsize=(9,0.5),ret_id=False):
    
    
    corr_mat = df.corr().round(2);shape = corr_mat.shape[0]
    corr_mat = corr_mat.transpose()
    corr = corr_mat.loc[:, df.columns == traget].transpose().copy()
    
    if(ret_id is False):
        f,ax = plt.subplots(figsize=figsize)
        sns.heatmap(corr,vmin=-0.3,vmax=0.3,center=0,
                   cmap=cmap,square=False,lw=2,annot=True,cbar=False)
        plt.title(f'Feature Correlation to {target}')
        
    if(ret_id):
        return corr
    
    
"""Correlation plot w/Plotly"""
def plotlyoff_corr(corr,size=None):
    
    xcols = corr.columns.tolist();ycols = xcols
    if(size is None):
        width = 700; height = 500
    else:
        width = size[0];height = size[1]
        
    layout = dict(
        width = width,height = height,
        yaxis = dict(tickangle=-30,side = 'left'),
        xaxis = dict(tickangle=-30,side = 'top'))
    fig = ff.create_annotated_heatmap(
        z=corr.values,x= xcols,y= ycols,
        colorscale='viridis',showscale=False)
    fig['layout'].update(layout)
    fig.update_layout(margin={"r":0,"t":60,"l":0,"b":0})
    
    return py.iplot(fig)

""" Draw a Bivariate Seaborn Pairgrid /wKDE density w/ """
def snsPairGrid(df):
    
    """Plots a Seaborn Pairgrid w/KDE & scatter plot of df features"""
    g = sns.PairGrid(df,diag_sgarey=False)
    g.fig.set_size_inches(14,13)
    g.map_diag(sns.kdeplot, lw=2) # draw kde approximation on the diagonal
    g.map_lower(sns.scatterplot,s=15,edgecolor="k",linewidth=1,alpha=0.4) # Scattered plot on lower half
    g.map_lower(sns,kdeplot,cmap='plasma',n_levels=10) # kde approximation on lower half
    plt.tight_layout()
    
"""Basic Transformer"""
class transformer(BaseEstimator,TransformerMixin):
    
    
    def __init__(self,drop_nan=False,select_dtype=False,show_nan=False,title='Title',show_counts=False,figsize=None):
        self.drop_nan = drop_nan
        self.select_dtype = select_dtype
        self.show_nan = show_nan
        self.title = title
        self.show_counts = show_counts
        self.figsize = figsize
        
    # Apply Fit
    def fit(self,X,y=None):
        return self
    
    # Apply Some Transformationto the FeatureMatrix
    def transform(self,X):
        
        
        # show NaN % in DataFrame
        if(self.show_nan):
            
            fig, ax = plt.subplots(figsize = self.figsize)
            nan_val = (X.isnull().sum()/len(x)*100).sort_values(ascending = False)
            cmap = sns.color_palette("plasma")
            for i in ['top', 'right', 'bottom', 'left']:
                ax.spines[i].set_color('black')
            ax.spines['top'].set_visible(True);ax.spines['right'].set_visible(False)
            ax.spines['bottom'].set_visible(False);ax.spines['left'].set_visible(False)
            plt.title(self.title);ax.grid(ls='--',alpha = 0.9);plt.show()
            return
        
        """ Plot df.value_counts"""
        if(self.show_counts):
            
            tdf = X.value_counts()
            cmap = sns.color_palette("plasma")
            fig, ax = plt.subplots(figsize = self.figsize)
            for i in ['top', 'right', 'bottom', 'left']:
                ax.spines[i].set_color('black')
            ax.spines['top'].set_visible(True);ax.spines['right'].set_visible(False)
            ax.spines['bottom'].set_visible(False);ax.spines['left'].set_visible(False)
            sns.barplot(tdf.index,tdf.values,edgecolor='k',palette='rainbow',ax=ax)
            plt.title(self.title);ax.grid(ls='--',alpha = 0.9);plt.show()
            
            
        """ Drop ALL NAN values in DataFrame"""
        if(self.drop_nan):
            X = X.dropna(); print(X,shape) # drop NaNvalues in df
            return X
        
        """ Split DataFrame into Numerical/Object Features"""
        if(self.select_dtype):
            X1 = X.select_dtype(include=['float64', 'int64'])  # return on ly numerical features from df
            X2 = X.selcet_dtype(exclude=['float64', 'int64'])
            return X1,X2
            
            
def function(ldf,id=None):
    
    if(id is 'boxplot'):
        # 1. Univariate Boxplots
        fig, axs = plt.subplots(ncols=5, nrow=3, figsize=(14,4))
        index = 0
        axs = axs.flatten()
        for k,v in X.items():
            
            
            flierprops = dict(maker='0', mfc='k', ms=3, ls='none', mec='k')
            ax = sns.boxplot(x=k, data=ldf, orient='h', flierprops=flierprops, ax=axs[index], width=.5)
            index +=1
        plt.tight_layout()
        
    elif(id is 'outiers'):
        # 2. Define Outlier
        for k, v in ldf.items():
            q1 = v.quantile(0.25); q3 = v.quantile(0.75); irq = q3- q1
            v_col = v[(v <= q1 - 1.5 * irq) | (v >= q3 + 1.5 * irp)]
            perc = np.shape(v_col)[0] * 100.0 / np.shape(ldf)[0]
            print("Column %s outliers = %.2f%%" %  (k, perc))
            
    elif(id is 'histograms'):
        # 3. Histograms
        fig, axs = plt.subplots(ncols=5, nrows=3, figsize=(14,8))
        index = 0
        axs = axs.flatten()
        for k,v in ldf.items():
            sns.histplot(v,ax=axs[index],bins=20)
            index += 1
        plt.tight_layout()
        
    elif(id is 'correlation'):
        # 4. Correlation Matrix
        cmap = sns.choose_diverging_palette(220, 10, as_cmap=True)
        # Bivariate Correlation Matrix
        def corrMat(df,id=False):
            
            
            corr_mat = df.corr().round(2)
            f, ax = plt.subplots(figsize=(9,7))
            mask = np.triu(np.ones_like(corr_mat, dtype=np.bool))
            mask = mask[1:,:-1]
            corr = corr_mat.iloc[1:,-1].copy()
            sns.heatmap(corr,mask=mask,vmin=-0.3,vmax=0.3,center=0,camp='Blue',square=False,lw=2,annot=True,cbar=False)
            
        corrMat(ldf)
        
""" Draw a Bivariate Seaborn Pairgrid /w KDE density w/ """
def snaPairGrid(df):
    
    """ Plot a Seaborn Pairgrid w/ KDE & scatter plot of dffeatures"""
    g = sns.PairGrid(df,diag_share=False)
    g.fig.set_size_inches(14,13)
    g.map_diag(sns.kdeplot, lw=2)  # draw kde approximation on the diagonal
    g.map_lower(sna.scatterplot,s=15,edgecolor="k",linewidth=1, alpha=0.4)  # scattered ploton lower half
    g.map_lower(sns.kdeplot,cmap='plasma',n_levels=10)  # kde approximation on lower half
    plt.tight_layout()
    

# Class to visualise Things Only
class visualise(BaseEstimator, TransformerMixin):
    
    
    def __init__(self,vis_mean_AUD=None,model_name=None,target=None,eval_fi=False,vis_box=False,
                 vis_compare_mean_AUD=False,vis_compare_bar=False,top_scat=50,top_bar=None,option=False):
        self.vis_mean_AUD = vis_mean_AUD # [T/F] triggrt for mean AUD visualisation
        self.model_name = model_name # define which model plot in vis_mean_AUD
        self.target = target
        self.vis_box = vis_box # boxplots for univariate analysis
        self.vis_compare_mean_AUD = vis_compare_mean_AUD
        self.top_scat = top_scat # export only top n number of suburbs for scatter matrix
        self.top_bar = top_bar # show only top cases for bar sort
        self.vis_compare_bar = vis_compare_bar # compare two bar plots (general)
        self.option = option
        
    @staticmethod
    def corrMat2(df,target='demand',figsize=(9,0.5),ret_id=False):
        
        corr_mat = df.corr().round(2);shape = corr_mat.shape[0]
        corr_mat = corr_mat.transpose()
        corr = corr_mat.loc[:,df.columns == target].transpose().copy()
        
        if(ret_id is False):
            f, ax = plt.subplots(figsize=figsize)
            sns.heatmap(corr,vmin=-0.3,vmax=0.3,center=0,
                       cmap=cmap,square=False,lw=2,annot=True,cbar=False)
            plt.title(f'Feature Correlation to {target}')
            
        if(ret_id):
            return corr
        
    def fit(self):
        return self
    
    # X -> Numerical (feature matrix + target variable)
    def transform(self,X):
        
        if(self.option is 'histogram'):
            vdf_perth1_num,_ = transformer(select_dtype=True).transform(X=X)
            vdf_perth1_num.hist(bins=60, figsize=(20,15));plot.show()
            
        """ show suburb based mean price prediction/price"""
        if(self.vis_mean_AUD is not None):
            
            
            # Combine Numerical & Object Features
            if(self.vis_mean_AUD is 'error'):
                tdfx = X.groupby(['suburb']).mean().sort_values(by=self.model_name+'_error',ascending=False)
                
                if(self.top_bar is not None):
                    dfx = X.groupby(['suburb']).mean().sort_values(by=self.model_name+'_error',ascending=False)[:self.top_bar]
                else:
                    dfx = tdfx
                fig = px.bar(dfx,x=dfx.index, y=[self.model_name+'_error'],template='plotly_white',height=400)
                fig.update_layout(barmode="group",title='Suburb Based Mean Error | (y_pred-y_target) |',showlegend=True)
            
            elif(self.vis_mean_AUD is 'value'):
                tdfx = X.groupby(['suburb']).mean().sort_values(by=self.model_name+'_error',ascending=False)
                if(self.top_bar is not None):
                    dfx = X.groupby(['suburb']).mean().sort_values(by=self.model_name+'_error',ascending=False)[:self.top_bar]
                else:
                    dfx = tdfx
                    
                fig = px.bar(dfx,x=dfx, y=[self.model_name,self.target],template='plotly_white',height=400)
                fig.update_layout(barmode="group",title='Suburb-Based Mean Prediction/Target Variable',showlegend=True)
                
            # Used for Scatter Matrix Import
            fig.show() # stack/overlay/group
            tdfx.loc[:,'group_id2'] = 0
            tdfx.loc[:self.top_scat,'group_id2'] = 1
            
            return tdfx
        """ Compare Mean Groupby Buburb Error Bars """
        # X -> List of Pandas DataFrame
        if(self.vis_compare_mean_AUD):
            
            dfx1 = X[0].groupby(['suburb']).mean().sort_values(by=self.model_name+'_error',ascending=False)
            dfx2 = X[1].groupby(['suburb']).mean().sort_values(by=self.model_name+'_error',ascending=False)
            dfx1.rename(columns = {self.model_name+'_error':'A'}, inplace = True)
            dfx2.rename(columns = {self.model_name+'_error':'B'}, inplace = True)
            dfx_all = pd.connect([dfx1['A'],dfx2['B']],axis=1)
            fig = px.bar(dfx_all,x=dfx_all.index, y=['A','B'],template='plotly_white',height=400)
            fig.update_layout(barmode="group",title='Suburb Based Mean Error |(y_pred-y_target)|',showlegend=True)
            fig.show()
        
        """ Compare Two Error Bars (general1)"""
        # X -> List of Pandas DataFrames
        if(self.vis_compare_bar):
            
            X[0].rename(columns = {self.model_name+'_error':'A'}, inplace = True)
            X[1].rename(columns = {self.model_name+'_error':'B'}, inplace = True)
            dfx_all = pd.concat([dfx1['A'],dfx2['B']],axis=1)
            fig = px.bar(dfx_all,x=dfx_all.index, y=['A','B'],template='plotly_white',height=400)
            fig.update_layout(barmode="group",title='Suburb Based Mean Error |(y_pred-y_target)|',showlegend=True)
            fig.show()
            
        """ Plot Univariate Boxplots for Data Distribution """
        if(self.vis_box):
            
            lX,_ = transformer(select_dtype=True).transform(X=X)
            fig,axs = plt.subplots(nclos=5,nrows=3,figsize=(14,4))
            index = 0
            axs = axs.flatten()
            for k,v in lX.items():
                flierprops = dict(marker='o',mfc='k',ls='none',mec='k')
                ax = sns.boxplot(x=k,data=lX,orient='h',flierprops=flierprops,
                                ax=axs[index],which=0.5)
                index += 1
            ply.tight_layout()
             
        if(self.option is 'outliers'):
            
    #      2. Define Outliers
            lX,_ = transformer(select_dtype=True).transformer(X=X)
            for k, v in lX.items():
                q1 = v.quantile(0.25); q3 = v.quantile(0.75); irq = q3 - q1
                v_col = v[(v <= q1 - 1.5 * irq) | (v >= q3 + 1.5 * irq)]
                perc = np.shape(v_col)[0] * 100.0 / np.shape(lX)[0]
                print("Column %s outliers = %.2f%%" % (k, perc))
    

# 1 | INTRODUCTION

1.1 | NOTEBOOK AIM

* I've decided to split the notebook into two parts **(Ⅰ) EDA notebook(this)** & **(Ⅱ) **ML models to predict price****.

* The aim of this article is to build some models that can predict **Perth**(located in Western Australia) housing prices based on a set of scrapped features made available in the **Perth Housing Dataset**.


**PART Ⅰ - EDA**
* Having learned how to use Plotly for geographic data analysis, thanks to Andrey Shtrauss. We will take a look at suburb based data as well. Let's see what we can learn about the dtaset, whilst we try to reach our goal of preciting the target variable price.
* The division is mainly done due to the heavy load the plots have on the notebook loading performance.

**PART Ⅱ - PREDICTION MODELS**
* Let's also used our own sklearn compatible calss, here has asimplistic **XGBoost** models (tree ensemble approaches) are used, which is one of the morepowerful models, and it might be interesting to see how well it perform compared to the XGBoost Library as well.


# 1.2 | PERTH HOUSING DATASET

* As far as Perth Housing Dataset goes, the current dataset is relatively extensive from an information point of view, we would probably benefit from some form of **geotagging** especially if we had address coordinates, however we will limit ourselves to **suburb** locations only, which will be extrtacted from a separate dataset.

In [3]:
df_perth0 = pd.read_csv('/kaggle/input/perth-house-prices/all_perth_310121.csv')
df_perth0.columns = map(str.lower, df_perth0.columns)
df_perth0 = df_perth0.drop(['longitude', 'latitude'],axis=1)
df_perth0.rename({'cbd_dist':'CBD_dist'},axis=1,inplace=True)
df_perth0.columns

Index(['address', 'suburb', 'price', 'bedrooms', 'bathrooms', 'garage',
       'land_area', 'floor_area', 'build_year', 'CBD_dist', 'nearest_stn',
       'nearest_stn_dist', 'date_sold', 'postcode', 'nearest_sch',
       'nearest_sch_dist', 'nearest_sch_rank'],
      dtype='object')

As far as these features are concerned, they are quite self explanatory:

* address: Physical address of the property (we will set to index)
* suburb: Specific locality in Perth; a list of all Perth suburb can be found here: https://www.homely.com.au/find-suburb-by-region/perth-greater-western-australia
* price: Price at which a property was sold (AUD)
* bedrooms: Number of bedrooms
* bathrooms: Number of bathrooms
* garage: Number of garage places
* land_area: Total land area (m^2)
* build_year: Year in which the property was built
* CBD_dist: Distance from the centre of Perth (m)
* nearst_stn: The nearest public transport station from the property
* nearest_stn_dst: The nearest station distance (m)
* date_sold: Month & Year i which th property was sold

# 1.3 | ADDITIONAL DATASET w/LATITUDE & LONGITUDE
* Aside from **CBD_dist** (distance to some point in the centre of Perth), we don't have other geographical information; two identical **CBD_dist** values don't necessarily mean they are located near each other, which is why even rough estimates of GPS locatons can be beneficial to include in order to improve our modoels.

* I have loaded some data which contains some approximate GPS coodinates for each suburb (averaged coordinates within a suburb), ideally each property would have been more useful.

* You can download the dataset containing Australian Postodes & create you own, the **long,lat** values can be extracted based on two conditions **state == WA** & corresponding **suburbs**, which are included in column **locality**. 

In [4]:
df0 = pd.read_csv('/kaggle/input/postcodes/australian_postcodes.csv')
df0.head()

,id,postcode,locality,state,long,lat,dc,type,status,sa3,...,MMM_2019,ced,altitude,chargezone,phn_code,phn_name,lgaregion,lgacode,electorate,electoraterating
0,230,200,ANU,ACT,149.11900,-35.277700,NaN,NaN,Updated 3-Dec-2022,NaN,...,1.0,NaN,NaN,N2,NaN,NaN,Unincorporated ACT,89399.0,Durack,NaN
1,21820,200,Australian National University,ACT,149.11890,-35.277700,NaN,NaN,Updated 3-Dec-2022,NaN,...,1.0,NaN,NaN,N2,NaN,NaN,Unincorporated ACT,89399.0,Durack,NaN
2,232,800,DARWIN,NT,130.83668,-12.458684,NaN,NaN,Updated 3-Dec-2022,70101.0,...,2.0,NaN,NaN,NT1,PHN701,Northern Territory,Darwin Waterfront Precinct,71150.0,Solomon,Inner Metropolitan
3,24049,800,DARWIN CITY,NT,130.83668,-12.458684,NaN,NaN,Updated 3-Dec-2022,70101.0,...,2.0,NaN,NaN,NT1,PHN701,Northern Territory,Darwin Waterfront Precinct,71150.0,Solomon,Inner Metropolitan
4,233,801,DARWIN,NT,130.83668,-12.458684,NaN,NaN,Updated 3-Dec-2022,70101.0,...,2.0,NaN,NaN,NT1,PHN701,NaN,Darwin Waterfront Precinct,71150.0,Lingiari,Rural


In [5]:
df_gps = pd.read_csv('/kaggle/input/postcodes/australian_postcodes.csv')
df_gps.columns

Index(['id', 'postcode', 'locality', 'state', 'long', 'lat', 'dc', 'type',
       'status', 'sa3', 'sa3name', 'sa4', 'sa4name', 'region', 'Lat_precise',
       'Long_precise', 'SA1_MAINCODE_2011', 'SA1_MAINCODE_2016',
       'SA2_MAINCODE_2016', 'SA2_NAME_2016', 'SA3_CODE_2016', 'SA3_NAME_2016',
       'SA4_CODE_2016', 'SA4_NAME_2016', 'RA_2011', 'RA_2016', 'MMM_2015',
       'MMM_2019', 'ced', 'altitude', 'chargezone', 'phn_code', 'phn_name',
       'lgaregion', 'lgacode', 'electorate', 'electoraterating'],
      dtype='object')

In [6]:
df_perth = pd.concat([df_perth0,df_gps[['long','lat']]],axis=1)
df_perth.head()

,address,suburb,price,bedrooms,bathrooms,garage,land_area,floor_area,build_year,CBD_dist,nearest_stn,nearest_stn_dist,date_sold,postcode,nearest_sch,nearest_sch_dist,nearest_sch_rank,long,lat
0,1 Acorn Place,South Lake,565000,4,2,2.0,600,160,2003.0,18300,Cockburn Central Station,1800,09-2018\r,6164,LAKELAND SENIOR HIGH SCHOOL,0.828339,NaN,149.11900,-35.277700
1,1 Addis Way,Wandi,365000,3,2,2.0,351,139,2013.0,26900,Kwinana Station,4900,02-2019\r,6167,ATWELL COLLEGE,5.524324,129.0,149.11890,-35.277700
2,1 Ainsley Court,Camillo,287000,3,1,1.0,719,86,1979.0,22600,Challis Station,1900,06-2015\r,6111,KELMSCOTT SENIOR HIGH SCHOOL,1.649178,113.0,130.83668,-12.458684
3,1 Albert Street,Bellevue,255000,2,1,2.0,651,59,1953.0,17900,Midland Station,3600,07-2018\r,6056,SWAN VIEW SENIOR HIGH SCHOOL,1.571401,NaN,130.83668,-12.458684
4,1 Aman Place,Lockridge,325000,4,1,2.0,466,131,1998.0,11200,Bassendean Station,2000,11-2016\r,6054,KIARA COLLEGE,1.514922,NaN,130.83668,-12.458684


In [7]:
df_perth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33656 entries, 0 to 33655
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   address           33656 non-null  object 
 1   suburb            33656 non-null  object 
 2   price             33656 non-null  int64  
 3   bedrooms          33656 non-null  int64  
 4   bathrooms         33656 non-null  int64  
 5   garage            31178 non-null  float64
 6   land_area         33656 non-null  int64  
 7   floor_area        33656 non-null  int64  
 8   build_year        30501 non-null  float64
 9   CBD_dist          33656 non-null  int64  
 10  nearest_stn       33656 non-null  object 
 11  nearest_stn_dist  33656 non-null  int64  
 12  date_sold         33656 non-null  object 
 13  postcode          33656 non-null  int64  
 14  nearest_sch       33656 non-null  object 
 15  nearest_sch_dist  33656 non-null  float64
 16  nearest_sch_rank  22704 non-null  float6

# 1.4 | DATA ASSEMBLY & CLEANING
* The scrapped data needs quite a bit of cleaning; containing ocassional errors due to incorrect data, as well as standard NaN missing data.

* Let's do some more obvious data cleaning here, dealing with **missing data** & **repetitive addresses**.

In [8]:
# Some Data Cleaning
df_perth.drop_duplicates(subset=['address'],inplace=True) # Some address actually have multiple entries
df_perth.index = df_perth['address'] # set dataframe index, since it is not really a useful feature 
del df_perth['address'] # let's also delete the column

In [9]:
""" Remove Missing Data """
#transformer(show_nan=True,figsize=(8,3),title='Feature (NaN) %').transform(X=df_perth)

' Remove Missing Data '